In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
m = d['manager1']
df = pd.concat([x.get_rideloop_experience_log_df() for x in m.agents])